Traning model

In [ ]:
import tensorflow as tf  # type: ignore
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers   # type: ignore

In [ ]:
Datadirectory = "test_data/"                     #dataset path
Classes = ["closed_eyes" ,"open_eyes"]           #dataset path
for category in Classes:
    path= os.path.join(Datadirectory,category)
    for img in os.listdir(path):
        img_array=cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
        backtorgb = cv2.cvtColor(img_array,cv2.COLOR_GRAY2RGB)
         
img_size=224
training_Data = []
def create_training_Data():
   for category in Classes:
       path = os.path.join(Datadirectory, category)
       class_num = Classes.index(category) # 0 1,
       for img in os.listdir(path):
           try:
               img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
               backtorgb = cv2.cvtColor(img_array,cv2.COLOR_GRAY2RGB)
               new_array = cv2.resize(backtorgb, (img_size, img_size))
               training_Data.append([new_array,class_num])
           except Exception as e:
               pass
create_training_Data()

import random
random.shuffle(training_Data)
x=[]
y=[]
for features, label in training_Data:
    x.append(features)
    y.append(label)
x=np.array(x).reshape(-1,img_size,img_size,3)

x=x/255.0;
y=np.array(y)

import pickle
pickle_out = open("x.pickle","wb")
pickle.dump(x,pickle_out)
pickle_out.close()
pickle_out = open("y.pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close()

model = tf.keras.applications.mobilenet.MobileNet()
base_input =model.layers[1].input
base_output = model.layers[-4].output
Flat_layer =layers.Flatten()(base_output)
final_output = layers.Dense(1)(Flat_layer)
final_output = layers.Activation('sigmoid')(final_output)
new_model = keras.Model(inputs = base_input , outputs = final_output)

new_model.compile(loss = "binary_crossentropy",optimizer = "adam",metrics=["accuracy"])
new_model.fit(x,y,epochs = 1,validation_split =0.1)
new_model.save("my_model.h5")

Testing model (input from given dataset)

In [ ]:
new_model = keras.models.load_model("my_model.h5")
img_array=cv2.imread("test_data\closed_eyes\s0003_00003_0_0_0_0_0_01.png",cv2.IMREAD_GRAYSCALE)   #test image path
img_size=224
backtorgb=cv2.cvtColor(img_array,cv2.COLOR_GRAY2RGB)
new_array=cv2.resize(backtorgb,(img_size,img_size))
x_input = np.array(new_array).reshape(1,img_size,img_size,3)
plt.imshow(new_array)

In [ ]:
x_input=x_input/255.0
prediction = new_model.predict(x_input)
prediction
if prediction <0.5:
    print("close")
else:
    print("open")    

Testing model (input from outside )

In [ ]:
new_model = keras.models.load_model("my1_model.h5")
img = cv2.imread("open_face.jpg")                    #test image path
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

In [ ]:
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")
gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
eyes = eye_cascade.detectMultiScale(gray,1.1,4)
for(x,y,w,h) in eyes:
    cv2.rectangle(img,(x,y), (x+w,y+h),(0,255,0),2)
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))    

In [ ]:
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")
gray= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
eyes= eye_cascade.detectMultiScale(gray, 1.1,4)
for x,y,w,h in eyes:
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
eyess =eye_cascade.detectMultiScale(roi_gray)
if len(eyess) == 0:
    print("eyes are not detected")
else:
    for (ex,ey,ew, eh) in eyess:
        eyes_roi = roi_color[ey:ey+eh,ex:ex+ew]
plt.imshow(cv2.cvtColor(eyes_roi,cv2.COLOR_BGR2RGB))        

In [ ]:
final_image  =cv2.resize(eyes_roi,(224,224))
final_image = np.expand_dims(final_image,axis=0)
final_image = final_image/255.0
new_model.predict(final_image)
if prediction <0.5:
    print("close")
else:
    print("open")    
    